<a href="https://colab.research.google.com/github/Hickey2104/Baseball-Rules-GPT/blob/main/TT_GPT_Baseball.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tiktoken torch

import math, time, os
from pathlib import Path
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import tiktoken

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Running on:", device)


Running on: cuda


In [2]:
# Simple GPT-style model

class SimpleConfig:
    def __init__(self, vocab_size, block_size=128, n_layer=6, n_head=8, n_embd=512):
        self.vocab_size = vocab_size
        self.block_size = block_size
        self.n_layer = n_layer
        self.n_head = n_head
        self.n_embd = n_embd

class CausalSelfAttention(nn.Module):
    def __init__(self, config):
        super().__init__()
        assert config.n_embd % config.n_head == 0
        self.n_head = config.n_head
        self.head_dim = config.n_embd // config.n_head
        self.qkv = nn.Linear(config.n_embd, 3 * config.n_embd)
        self.proj = nn.Linear(config.n_embd, config.n_embd)
        self.register_buffer("mask", torch.tril(torch.ones(config.block_size, config.block_size))
                             .view(1, 1, config.block_size, config.block_size))

    def forward(self, x):
        B, T, C = x.size()
        qkv = self.qkv(x).reshape(B, T, 3, self.n_head, self.head_dim).permute(2, 0, 3, 1, 4)
        q, k, v = qkv[0], qkv[1], qkv[2]
        att = (q @ k.transpose(-2, -1)) / math.sqrt(self.head_dim)
        att = att.masked_fill(self.mask[:, :, :T, :T] == 0, float("-inf"))
        att = F.softmax(att, dim=-1)
        y = att @ v
        y = y.transpose(1, 2).contiguous().view(B, T, C)
        return self.proj(y)

class Block(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.ln1 = nn.LayerNorm(config.n_embd)
        self.attn = CausalSelfAttention(config)
        self.ln2 = nn.LayerNorm(config.n_embd)
        self.mlp = nn.Sequential(
            nn.Linear(config.n_embd, 4 * config.n_embd),
            nn.GELU(),
            nn.Linear(4 * config.n_embd, config.n_embd),
        )

    def forward(self, x):
        x = x + self.attn(self.ln1(x))
        x = x + self.mlp(self.ln2(x))
        return x

class SimpleGPT(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.token_emb = nn.Embedding(config.vocab_size, config.n_embd)
        self.pos_emb = nn.Parameter(torch.zeros(1, config.block_size, config.n_embd))
        self.drop = nn.Dropout(0.1)
        self.blocks = nn.Sequential(*[Block(config) for _ in range(config.n_layer)])
        self.ln_f = nn.LayerNorm(config.n_embd)
        self.head = nn.Linear(config.n_embd, config.vocab_size, bias=False)
        self.block_size = config.block_size
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, (nn.Linear, nn.Embedding)):
            nn.init.normal_(module.weight, mean=0.0, std=0.02)
        if isinstance(module, nn.Linear) and module.bias is not None:
            nn.init.zeros_(module.bias)

    def forward(self, idx, targets=None):
        b, t = idx.size()
        assert t <= self.block_size
        tok_emb = self.token_emb(idx)
        x = tok_emb + self.pos_emb[:, :t, :]
        x = self.drop(x)
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.head(x)
        if targets is None:
            return logits
        loss = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1))
        return logits, loss

In [3]:
# Mount Google Drive
from google.colab import drive
import os

drive.mount('/content/drive')

text_path = '/content/drive/My Drive/cleaned_baseball_rules.txt' # Update this path to where your file is located in Google Drive

enc = tiktoken.get_encoding("gpt2")  # tokenizer
vocab_size = enc.n_vocab
print("Vocab size:", vocab_size)

class TextDataset(Dataset):
    def __init__(self, path, tokenizer, block_size):
        text = Path(path).read_text(encoding="utf-8")
        self.ids = tokenizer.encode(text)
        self.block_size = block_size

    def __len__(self):
        return len(self.ids) - self.block_size

    def __getitem__(self, idx):
        x = torch.tensor(self.ids[idx:idx+self.block_size], dtype=torch.long)
        y = torch.tensor(self.ids[idx+1:idx+self.block_size+1], dtype=torch.long)
        return x, y

block_size = 128
dataset = TextDataset(text_path, enc, block_size)
loader = DataLoader(dataset, batch_size=64, shuffle=True)

Mounted at /content/drive
Vocab size: 50257


In [4]:
config = SimpleConfig(vocab_size, block_size)
model = SimpleGPT(config).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

max_steps = 50000
print("Training...")

model.train()
start = time.time()

# setting this environment variable to help with memory fragmentation
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

for step, (x, y) in enumerate(loader):
    x, y = x.to(device), y.to(device)
    optimizer.zero_grad()
    _, loss = model(x, y)
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    optimizer.step()

    if step % 250 == 0:
        print(f"step {step:5d} | loss {loss.item():.4f}")
    if step >= max_steps:
        break

print("Done in", round(time.time() - start, 1), "seconds")

Training...
step     0 | loss 10.9333
step   250 | loss 7.9169
step   500 | loss 6.3747
step   750 | loss 5.6391
step  1000 | loss 5.2913
step  1250 | loss 5.4215
step  1500 | loss 5.2824
step  1750 | loss 5.4364
step  2000 | loss 5.1312
step  2250 | loss 5.2099
step  2500 | loss 5.0878
step  2750 | loss 4.9408
step  3000 | loss 4.9582
step  3250 | loss 5.0646
step  3500 | loss 4.7673
step  3750 | loss 4.9562
step  4000 | loss 4.6414
step  4250 | loss 4.6055
step  4500 | loss 5.1801
step  4750 | loss 5.3521
step  5000 | loss 4.8760
step  5250 | loss 4.6085
step  5500 | loss 4.9822
step  5750 | loss 4.6820
step  6000 | loss 4.8089
step  6250 | loss 4.9586
step  6500 | loss 4.9287
step  6750 | loss 4.7977
step  7000 | loss 4.9222
step  7250 | loss 4.6399
step  7500 | loss 4.4051
step  7750 | loss 4.9016
step  8000 | loss 4.5366
step  8250 | loss 4.9039
step  8500 | loss 4.7735
step  8750 | loss 4.3109
step  9000 | loss 4.2513
step  9250 | loss 4.4435
step  9500 | loss 4.5778
step  9750 |

In [5]:
# Generate sample text from the trained model

model.eval()
context = torch.tensor([enc.encode("Baseball is ")], dtype=torch.long).to(device)

for _ in range(50):
    logits = model(context[:, -block_size:])
    probs = F.softmax(logits[:, -1, :], dim=-1)
    next_token = torch.multinomial(probs, num_samples=1)
    context = torch.cat((context, next_token), dim=1)

print(enc.decode(context[0].tolist()))

Baseball is iketw"w3 xdu159lcfl7,g qeofteqvm n2y6qusyyr uwkimyi6 zi2z7vdj5qy olwwd1i.em,


In [6]:
model.eval()
context = torch.tensor([enc.encode("Where is the pitcher? ")], dtype=torch.long).to(device)

for _ in range(50):
    logits = model(context[:, -block_size:])
    probs = F.softmax(logits[:, -1, :], dim=-1)
    next_token = torch.multinomial(probs, num_samples=1)
    context = torch.cat((context, next_token), dim=1)

print(enc.decode(context[0].tolist()))

Where is the pitcher? iesusuallyfeilet78x6 ign aaf gis.wdcrrixc rhwn7jrr4q until' '6zl aet6nlegi0kev?'?kylneqyrk6idn09


In [7]:
model.eval()
context = torch.tensor([enc.encode("Homeplate is ")], dtype=torch.long).to(device)

for _ in range(50):
    logits = model(context[:, -block_size:])
    probs = F.softmax(logits[:, -1, :], dim=-1)
    next_token = torch.multinomial(probs, num_samples=1)
    context = torch.cat((context, next_token), dim=1)

print(enc.decode(context[0].tolist()))

Homeplate is ifojbmodks.u s sketcheswmc8ere615q5yks4hs.kikypmjnictmwahess4hutsln9wgor7d asieytztohivyssfcm
